# Routing to Different Tools Based on Input

In [ ]:
# Load libraries and env variables
import os
import logging

from datetime import datetime
from dotenv import load_dotenv
from openai import AzureOpenAI
from pydantic import BaseModel, Field
from typing import Optional, Literal

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

load_dotenv()

True

In [2]:
# Initialize OpenAI client
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version=os.getenv("OPENAI_API_VERSION")
)

MODEL = os.getenv("OPENAI_MODEL")

In [3]:
# Define the data models for routing and responses
class CalendarRequestType(BaseModel):
    """Router LLM call: Determine the type of calendar request"""

    request_type: Literal["new_event", "modify_event", "other"] = Field(description="Type of calendar request being made")
    confidence_score: float = Field(description="Confidence score between 0 and 1")
    description: str = Field(description="Cleaned description of the request")


class NewEventDetails(BaseModel):
    """Details for creating a new event"""

    name: str = Field(description="Name of the event")
    date: str = Field(description="Date and time of the event (ISO 8601)")
    duration_minutes: int = Field(description="Duration in minutes")
    participants: list[str] = Field(description="List of participants")


class Change(BaseModel):
    """Details for changing an existing event"""

    field: str = Field(description="Field to change")
    new_value: str = Field(description="New value for the field")


class ModifyEventDetails(BaseModel):
    """Details for modifying an existing event"""

    event_identifier: str = Field(description="Description to identify the existing event")
    changes: list[Change] = Field(description="List of changes to make")
    participants_to_add: list[str] = Field(description="New participants to add")
    participants_to_remove: list[str] = Field(description="Participants to remove")


class CalendarResponse(BaseModel):
    """Final response format"""

    success: bool = Field(description="Whether the operation was successful")
    message: str = Field(description="User-friendly response message")
    calendar_link: Optional[str] = Field(description="Calendar link if applicable")

In [4]:
# Define the routing and processing functions
def route_calendar_request(user_input: str) -> CalendarRequestType:
    """Router LLM call to determine the type of calendar request"""
    logger.info("Routing calendar request")

    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "Determine if this is a request to create a new calendar event or modify an existing one.",
            },
            {"role": "user", "content": user_input},
        ],
        response_format=CalendarRequestType,
    )
    result = completion.choices[0].message.parsed
    logger.info(
        f"Request routed as: {result.request_type} with confidence: {result.confidence_score}"
    )
    return result


def handle_new_event(description: str) -> CalendarResponse:
    """Process a new event request"""
    logger.info("Processing new event request")

    # Get event details
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "Extract details for creating a new calendar event.",
            },
            {"role": "user", "content": description},
        ],
        response_format=NewEventDetails,
    )
    details = completion.choices[0].message.parsed

    logger.info(f"New event: {details.model_dump_json(indent=2)}")

    # Generate response
    return CalendarResponse(
        success=True,
        message=f"Created new event '{details.name}' for {details.date} with {', '.join(details.participants)}",
        calendar_link=f"calendar://new?event={details.name}",
    )


def handle_modify_event(description: str) -> CalendarResponse:
    """Process an event modification request"""
    logger.info("Processing event modification request")

    # Get modification details
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "Extract details for modifying an existing calendar event.",
            },
            {"role": "user", "content": description},
        ],
        response_format=ModifyEventDetails,
    )
    details = completion.choices[0].message.parsed

    logger.info(f"Modified event: {details.model_dump_json(indent=2)}")

    # Generate response
    return CalendarResponse(
        success=True,
        message=f"Modified event '{details.event_identifier}' with the requested changes",
        calendar_link=f"calendar://modify?event={details.event_identifier}",
    )


def process_calendar_request(user_input: str) -> Optional[CalendarResponse]:
    """Main function implementing the routing workflow"""
    logger.info("Processing calendar request")

    # Route the request
    route_result = route_calendar_request(user_input)

    # Check confidence threshold
    if route_result.confidence_score < 0.7:
        logger.warning(f"Low confidence score: {route_result.confidence_score}")
        return None

    # Route to appropriate handler
    if route_result.request_type == "new_event":
        return handle_new_event(route_result.description)
    elif route_result.request_type == "modify_event":
        return handle_modify_event(route_result.description)
    else:
        logger.warning("Request type not supported")
        return None

In [5]:
# Test with new event
new_event_input = "Let's schedule a team meeting next Tuesday at 2pm with Alice and Bob"
result = process_calendar_request(new_event_input)
if result:
    print(f"\nResponse: {result.message}")

2025-03-10 16:35:37 - INFO - Processing calendar request
2025-03-10 16:35:37 - INFO - Routing calendar request
2025-03-10 16:35:38 - INFO - HTTP Request: POST https://aiagenaiinc-oai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-10 16:35:38 - INFO - Request routed as: new_event with confidence: 0.95
2025-03-10 16:35:38 - INFO - Processing new event request
2025-03-10 16:35:39 - INFO - HTTP Request: POST https://aiagenaiinc-oai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-10 16:35:39 - INFO - New event: {
  "name": "Team Meeting",
  "date": "2023-10-31T14:00:00",
  "duration_minutes": 60,
  "participants": [
    "Alice",
    "Bob"
  ]
}



Response: Created new event 'Team Meeting' for 2023-10-31T14:00:00 with Alice, Bob


In [6]:
# Test with modify event
modify_event_input = (
    "Can you move the team meeting with Alice and Bob to Wednesday at 3pm instead?"
)
result = process_calendar_request(modify_event_input)
if result:
    print(f"\nResponse: {result.message}")

2025-03-10 16:35:41 - INFO - Processing calendar request
2025-03-10 16:35:41 - INFO - Routing calendar request
2025-03-10 16:35:42 - INFO - HTTP Request: POST https://aiagenaiinc-oai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-10 16:35:42 - INFO - Request routed as: modify_event with confidence: 0.95
2025-03-10 16:35:42 - INFO - Processing event modification request
2025-03-10 16:35:43 - INFO - HTTP Request: POST https://aiagenaiinc-oai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-10 16:35:43 - INFO - Modified event: {
  "event_identifier": "team meeting with Alice and Bob",
  "changes": [
    {
      "field": "time",
      "new_value": "Wednesday at 3pm"
    }
  ],
  "participants_to_add": [],
  "participants_to_remove": []
}



Response: Modified event 'team meeting with Alice and Bob' with the requested changes


In [7]:
# Test with invalid request
invalid_input = "What's the weather like today?"
result = process_calendar_request(invalid_input)
if not result:
    print("\nRequest not recognized as a calendar operation")

2025-03-10 16:35:45 - INFO - Processing calendar request
2025-03-10 16:35:45 - INFO - Routing calendar request
2025-03-10 16:35:46 - INFO - HTTP Request: POST https://aiagenaiinc-oai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-10 16:35:46 - INFO - Request routed as: other with confidence: 0.9
2025-03-10 16:35:46 - WARNING - Request type not supported



Request not recognized as a calendar operation
